## Decoupling Tools from Agent and Other Dependencies

In the lessons, we have been writing tools that use an architectural concept that we haven’t discussed yet, the action_context. This pattern is helpful for supporting dependency injection and decoupling tools from the agent and other dependencies. Let’s explore how the action_context pattern works and why it’s valuable for building flexible and maintainable agent architectures.

Let’s take a look at an example of why we need dependency injection and how the action_context pattern can help. Consider building an AI agent that helps develop and review code. In a typical workflow, the agent first acts as an expert developer to write code based on requirements. Later, we want the agent to switch roles and act as a code reviewer, critiquing its own work. This seems straightforward at first, but it surfaces a crucial architectural challenge.

For the code review to be effective, the LLM needs to understand the full context of how the code was developed. What requirements were specified? What constraints were discussed? What alternatives were considered and rejected?

Let’s assume that all of this information exists in the conversation history stored in the agent’s memory. What we would like to do is allow the LLM to see the memory of what happened in the agent loop, which contains a history of the code generation and requirements, and use it to provide a more informed code review. Let’s attempt to implement this code review tool:

In [ ]:
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(code: str) -> str:
    """Review code quality and suggest improvements."""
    # But how do we access the conversation history?
    # We can't just import the agent instance - that would create tight coupling

    return prompt_expert(
        description_of_expert="""
        Senior software architect reviewing code quality
        """,
        prompt=f"Review this code:\n{code}"
    )

We face an immediate problem. Our tool needs access to the conversation history in the memory, but we can’t simply import the agent instance or directly access its memory. That would create tight coupling between the tools and the agent implementation, making our tools less reusable and harder to test.

What we need is a way to provide the tool with access to memory and other resources without creating direct dependencies. This is where the ActionContext pattern comes in.

The ActionContext serves as a container for any resources that tools might need during their execution:

In [ ]:
class ActionContext:
    def __init__(self, properties: Dict=None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory", None)

Now we can modify our tool to accept an ActionContext, allowing it to access the conversation history without knowing where it comes from:

In [ ]:
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    """Review code quality and suggest improvements."""
    # Get memory to understand the code's context
    memory = action_context.get_memory()

    # Extract relevant history
    development_context = []
    for mem in memory.get_memories():
        if mem["type"] == "user":
            development_context.append(f"User: {mem['content']}")
        # Hypotethical scenario where our agent includes the phrase "Here's the implementation" when it generates code
        elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
            development_context.append(f"Implementation Decision: {mem['content']}")

    # Create review prompt with full context
    review_prompt = f"""Review this code in the context of its development history:

Development History:
{'\n'.join(development_context)}

Current Implementation:
{code}

Analyze:
1. Does the implementation meet all stated requirements?
2. Are all constraints and considerations from the discussion addressed?
3. Have any requirements or constraints been overlooked?
4. What improvements could make the code better while staying within the discussed parameters?
"""

    generate_response = action_context.get("llm")
    return generate_response(review_prompt)

##Handling Session or Request-Specific Dependencies
This pattern of dependency injection through ActionContext becomes even more valuable when we need to handle authentication-specific information. Consider a tool that needs to update a project management system with the status of code reviews. This requires authentication, but we don’t want to hardcode credentials or configure them at the tool level.

Here’s how we can use ActionContext to handle authentication:

In [ ]:
@register_tool(
    description="Update code review status in project management system",
    tags=["project_management"]
)
def update_review_status(action_context: ActionContext,
                        review_id: str,
                        status: str) -> dict:
    """Update the status of a code review in the project system."""
    # Get the authentication token for this specific request
    auth_token = action_context.get("auth_token")
    if not auth_token:
        raise ValueError("Authentication token not found in context")

    # Make authenticated request
    headers = {
        "Authorization": f"Bearer {auth_token}",
        "Content-Type": "application/json"
    }

    response = requests.post(
        f"https://...someapi.../reviews/{review_id}/status",
        headers=headers,
        json={"status": status}
    )

    if response.status_code != 200:
        raise ValueError(f"Failed to update review status: {response.text}")

    return {"status": "updated", "review_id": review_id}

When the agent runs a tool, it provides all necessary context through the ActionContext:



In [ ]:
def run(self, user_input: str, memory=None, action_context_props=None):
    """Execute the agent loop."""
    memory = memory or Memory()

    # Create context with all necessary resources
    action_context = ActionContext({
        'memory': memory,
        'llm': self.generate_response,
        # Request-specific auth
        **action_context_props
    })

    while True:
        prompt = self.construct_prompt(action_context, self.goals, memory)
        response = self.prompt_llm_for_action(action_context, prompt)
        result = self.handle_agent_response(action_context, response)

        if self.should_terminate(action_context, response):
            break

...
# Run the agent and create custom context for the action to
# pass to tools that need it
some_agent.run("Update the project status...",
               memory=...,
               # Pass request-specific auth token
               action_context_props={"auth_token": "my_auth_token"})

By using ActionContext, we’ve solved several key challenges:

- Tools can access conversation history without being coupled to the agent implementation
- Authentication and other request-specific information can be injected where needed
- Tools remain independent and testable since their dependencies are explicitly declared
- The agent can provide different contexts for different execution environments (development, production, testing)